# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG = or_suite.envs.env_configs.resource_allocation_foodbank_config(5)
# CONFIG = or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 100

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
#  'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
 'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
 'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[35.32146354 34.82342926 35.60643908 35.60462872 33.86564477]
 [43.1625433  42.27228204 42.09735355 43.13484842 41.40156445]
 [64.59694454 64.13501184 64.72398915 65.17613151 62.66874067]] [[ 619.84840311  589.11307591  561.98606976  583.14221513  527.47570332]
 [1014.03791607  922.29527899  881.63801835  900.8344631   933.14029223]
 [2743.60976587 2739.21570403 2871.28815007 2686.77163026 2761.33654738]]
Mean and variance endomwnets:
[[36.79063473 36.62856159 36.21216004 34.74872621 35.62584123]
 [43.25381407 44.17079193 43.27272857 41.82326674 42.76529222]
 [66.11117137 65.66326239 62.25431907 61.9071235  65.49413413]] [[ 651.75786011  626.92255854  623.73817886  576.42293999  614.57833337]
 [ 920.00431182  968.24246067  961.61214291  977.32266414  981.76924734]
 [2741.67804652 2563.82622333 2631.13082255 2531.1700942  2815.38005979]]
Mean and variance endomwnets:
[[36.66104844 34.47848764 33.4236911  37.64544109 34.14830365]
 [43.61125171 41.77030456 4

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = {'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG), \
                      'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG), \
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG), \
                      'Budget' : lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

Guardrail-0.5
Lower and Upper Solutions:
[[0.05469 0.05887 0.09567 0.06035 0.     ]
 [0.08916 0.08383 0.      0.08204 0.     ]
 [0.03893 0.03996 0.07222 0.04026 0.12533]]
[[0.09893 0.1065  0.17308 0.10917 0.     ]
 [0.1613  0.15164 0.      0.14841 0.     ]
 [0.07042 0.07229 0.13063 0.07283 0.22673]]
Writing to file data.csv
Guardrail-0.3
Lower and Upper Solutions:
[[0.05447 0.05859 0.09631 0.06004 0.     ]
 [0.08919 0.08371 0.      0.08187 0.     ]
 [0.03861 0.03978 0.07194 0.04013 0.12649]]
[[0.14211 0.1529  0.25186 0.15669 0.     ]
 [0.23288 0.21858 0.      0.21379 0.     ]
 [0.10088 0.10392 0.18764 0.10484 0.33028]]
Writing to file data.csv
Guardrail-0.25
Lower and Upper Solutions:
[[0.0547  0.05879 0.09537 0.06022 0.     ]
 [0.08894 0.08371 0.      0.08197 0.     ]
 [0.03897 0.03995 0.07184 0.04024 0.12509]]
[[0.1648  0.17721 0.28884 0.18158 0.     ]
 [0.26845 0.25274 0.      0.24748 0.     ]
 [0.11785 0.12074 0.21636 0.12159 0.37762]]
Writing to file data.csv
        Algorithm    

In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)